### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [8]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [22]:
def kmp_search(s, sub):

  p = [0] * len(sub)
  i=1
  j=0
  while i < len (sub):
    if sub[i] == sub[j]:
      p[i]=j+1
      i+=1
      j+=1
    else:
      if j==0:
        p[i]=0
        i+=1
      else:
        j = p[j-1]

  print(p)

  ss=0
  subb=0
  while ss < len(s):
    if s[ss] == sub[subb]:
      ss+=1
      subb+=1
      if subb == len(sub):
        return "We found it! " + str(ss-subb)
    else:
      if subb == 0:
        ss+=1
        if ss == len(s):
          return "There is no such substring"
      else:
        subb = p[subb-1]

#### Упрощенный алгоритм Бойера-Мура

In [13]:
def build_bad_char_table(pattern):
    table = defaultdict(lambda: -1)
    for i in range(len(pattern)):
        table[pattern[i]] = i
    return table


def simplified_boyer_moore(text, pattern):
    bad_char_table = build_bad_char_table(pattern)
    n, m = len(text), len(pattern)
    i = 0 

    while i <= n - m:
        j = m - 1  
        while j >= 0 and pattern[j] == text[i + j]:
            j -= 1

        if j < 0:
            return i
        else:
            bad_char_shift = bad_char_table[text[i + j]]
            shift = max(1, j - bad_char_shift)
            i += shift

    return -1

In [ ]:
s = input('Enter string: ')
sub = input('Enter substring: ')

user_input = input("Введите True (чувствительный к регистру) / False (нечувствительный): ").strip().lower()
if user_input == "true":
    value = True
elif user_input == "false":
    value = False
else:
    print("Ошибка: введите True или False")
    value = None


if value == False:
    s = s.lower()
    sub = sub.lower()

# kmp_search(s, sub)
simplified_boyer_moore(s, sub)

9

In [19]:
text = "abcabeabcabcabd"
pattern = "abcabd"

start = t.now()
index = kmp_search(text, pattern)
end = t.now()
print("Результат:", index)
print("Время:", end - start)

start = t.now()
index = simplified_boyer_moore(text, pattern)
end = t.now()
print("Результат:", index)
print("Время:", end - start)

start = t.now()
index = text.find(pattern)
end = t.now()
print("Результат:", index)
print("Время:", end - start)

[0, 0, 0, 1, 2, 0]
Результат: We found it! 9
Время: 0:00:00.000978
Результат: 9
Время: 0:00:00.000989
Результат: 9
Время: 0:00:00.001531


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [23]:
from collections import deque

GOAL = [1, 2, 3, 4,
        5, 6, 7, 8,
        9,10,11,12,
       13,14,15, 0]

MOVES = {
    'U': -4, 
    'D':  4,  
    'L': -1,  
    'R':  1   
}

def is_solvable(puzzle):
    inv = 0
    for i in range(len(puzzle)):
        for j in range(i + 1, len(puzzle)):
            if puzzle[i] and puzzle[j] and puzzle[i] > puzzle[j]:
                inv += 1
    row_from_bottom = 4 - (puzzle.index(0) // 4)
    if row_from_bottom % 2 == 0:
        return inv % 2 == 1
    else:
        return inv % 2 == 0

def get_neighbors(state):
    zero_pos = state.index(0)
    neighbors = []
    row, col = divmod(zero_pos, 4)
    for move, delta in MOVES.items():
        new_pos = zero_pos + delta
        new_row, new_col = divmod(new_pos, 4)
        if 0 <= new_pos < 16 and abs(row - new_row) + abs(col - new_col) == 1:
            new_state = state[:]
            new_state[zero_pos], new_state[new_pos] = new_state[new_pos], new_state[zero_pos]
            neighbors.append((new_state, state[new_pos]))  
    return neighbors

def solve(puzzle):
    if not is_solvable(puzzle):
        return []

    visited = set()
    queue = deque()
    queue.append((puzzle, []))

    while queue:
        state, path = queue.popleft()
        state_t = tuple(state)
        if state == GOAL:
            return path
        if state_t in visited:
            continue
        visited.add(state_t)
        for neighbor, moved in get_neighbors(state):
            queue.append((neighbor, path + [moved]))
    
    return []  


start = [1, 2, 3, 4,
         5, 6, 7, 8,
         9,10,11,12,
        13,15,14, 0]

result = solve(start)
print("Решение найдено!" if result else "Нерешаемо.")
print("Ходы:", result)

Нерешаемо.
Ходы: []


### Вывод